In [1]:
!pip install llama-index

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/2b/2b/0d4914384513dd15ea60a52a0f6b380b7dee2be7e2049e4c0986ed863305/llama_index-0.8.20-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=2.0.15 from https://files.pythonhosted.org/packages/91/2b/92aadcea86b9ebd681de0b6b2cbfa75193227e607893cfb5feea0cefc461/SQLAlchemy-2.0.20-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/3a/9f/b40e8e5be886143379000af5fc0c675352d59e82fd869d24bf784161dc77/fsspec-2023.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.3/752.3 kB 1.5 MB/s eta 0:00:001.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 2.7 MB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: sqlalchemy
 

## Setting API KEY

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

## build a simple vector store index

In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [6]:
print(query_engine.query("what is the culture of India"))

India's culture is known for its incredible diversity, with a variety of languages, religions, and traditions. It is often referred to as the "Land of Diversity" due to the numerous ethnic groups that call it home. The country's cultural landscape is heavily influenced by its religious diversity, with major religions such as Hinduism, Buddhism, Jainism, and Sikhism originating in India. Art and architecture are integral to Indian culture, with intricate temple sculptures, colorful frescoes, and stunning architectural wonders like the Taj Mahal. Indian cuisine is famous worldwide for its diverse flavors and regional specialties, with spices playing a central role in cooking. Traditional Indian clothing varies across regions, with sarees, dhotis, and kurta-pajamas being commonly worn. India is also renowned for its vibrant festivals, such as Diwali (Festival of Lights) and Holi (Festival of Colors), which are celebrated with enthusiasm and fervor. India's history dates back to the ancien

In [7]:
print(query_engine.query("List down Festivals of USA with its's date if possible"))

Independence Day - July 4th
Thanksgiving - Fourth Thursday in November
Halloween - October 31st
Christmas - December 25th
Labor Day - First Monday in September


In [9]:
print(query_engine.query("what is the language of Pakistan"))

The language of Pakistan is Urdu.


## Storing

In [10]:
index.storage_context.persist('countries-index')

## Loading Index

In [11]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='countries-index')
# load index
index = load_index_from_storage(storage_context)

In [12]:
query_engine_new = index.as_query_engine()

In [13]:
print(query_engine_new.query("List down Festivals of USA with its's date if possible"))

Independence Day - July 4th
Thanksgiving - Fourth Thursday in November
Halloween - October 31st
Christmas - December 25th
Labor Day - First Monday in September


# Customization

## parse documents into smaller chunks

In [14]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(chunk_size=1000)
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("List the Cuisines of England?")
print(response)

English cuisine includes classics like fish and chips, roast dinners, and afternoon tea. The country also has a diverse culinary scene influenced by immigration, with a variety of international restaurants.
